In [2]:
!! pip install selenium

['Collecting selenium',
 '  Downloading selenium-4.18.1-py3-none-any.whl.metadata (6.9 kB)',
 'Requirement already satisfied: urllib3<3,>=1.26 in ./venv/lib/python3.9/site-packages (from urllib3[socks]<3,>=1.26->selenium) (2.2.1)',
 'Collecting trio~=0.17 (from selenium)',
 '  Downloading trio-0.24.0-py3-none-any.whl.metadata (4.9 kB)',
 'Collecting trio-websocket~=0.9 (from selenium)',
 '  Downloading trio_websocket-0.11.1-py3-none-any.whl.metadata (4.7 kB)',
 'Requirement already satisfied: certifi>=2021.10.8 in ./venv/lib/python3.9/site-packages (from selenium) (2024.2.2)',
 'Requirement already satisfied: typing_extensions>=4.9.0 in ./venv/lib/python3.9/site-packages (from selenium) (4.10.0)',
 'Requirement already satisfied: attrs>=20.1.0 in ./venv/lib/python3.9/site-packages (from trio~=0.17->selenium) (23.2.0)',
 'Collecting sortedcontainers (from trio~=0.17->selenium)',
 '  Downloading sortedcontainers-2.4.0-py2.py3-none-any.whl.metadata (10 kB)',
 'Requirement already satisfie

In [8]:
!! pip install pandas

['Collecting pandas',
 '  Downloading pandas-2.2.1-cp39-cp39-macosx_11_0_arm64.whl.metadata (19 kB)',
 'Collecting numpy<2,>=1.22.4 (from pandas)',
 '  Downloading numpy-1.26.4-cp39-cp39-macosx_11_0_arm64.whl.metadata (61 kB)',
 '\x1b[?25l     \x1b━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━\x1b \x1b0.0/61.1 kB\x1b \x1b?\x1b eta \x1b-:--:--\x1b',
 '\x1b[2K     \x1b━━━━━━━━━━━━━━━━━━━━\x1b\x1b╺\x1b\x1b━━━━━━━━━━━━━━━━━━━\x1b \x1b30.7/61.1 kB\x1b \x1b2.4 MB/s\x1b eta \x1b0:00:01\x1b',
 '\x1b[2K     \x1b━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━\x1b \x1b61.1/61.1 kB\x1b \x1b1.6 MB/s\x1b eta \x1b0:00:00\x1b',
 '\x1b[?25hRequirement already satisfied: python-dateutil>=2.8.2 in ./venv/lib/python3.9/site-packages (from pandas) (2.9.0.post0)',
 'Collecting pytz>=2020.1 (from pandas)',
 '  Downloading pytz-2024.1-py2.py3-none-any.whl.metadata (22 kB)',
 'Collecting tzdata>=2022.7 (from pandas)',
 '  Downloading tzdata-2024.1-py2.py3-none-any.whl.metadata (1.4 kB)',
 'Requirement already satisfied: si

In [5]:
import pandas as pd
import selenium
import time
import re
from datetime import datetime
from selenium.common.exceptions import NoSuchElementException
print(f'{pd.__version__=}')
print(f'{selenium.__version__=}')
print(f'{re.__version__=}')



pd.__version__='2.2.1'
selenium.__version__='4.18.1'
re.__version__='2.2.1'


In [6]:
from selenium.webdriver.common.by import By

In [7]:
from selenium import webdriver
# # Open a website
# driver.get('https://www.google.com')
# #testing the webdriver

In [8]:
driver = webdriver.Firefox()

In [9]:
# #My Source
# mediamarkt_home = driver.get("https://www.mediamarkt.ch/de/category/_smartphone-680815.html?searchParams=&sort=&view=PRODUCTGRID&page=1")

In [12]:
phone_list = [] 
i = 1
while True:
    url = f"https://www.mediamarkt.ch/de/category/_smartphone-680815.html" + \
          f"?searchParams=&sort=&view=PRODUCTGRID&page={i}"
    
    try:
        driver.get(url)
    except WebDriverException as e:
        print(f"Error: {e}")
        break
    
    smartphones = driver.find_elements(By.CSS_SELECTOR, 
                                       'ul.products-grid > li')

    phone_list_page = []
    
    for phone in smartphones:
       
        #Info: brand, model, size, space
        info = phone.find_element(By.CSS_SELECTOR, "a.product-link").text
        brand_model = info.split("-")[0]
        brand, model = brand_model.split(" ", maxsplit=1)
        
        category_match = re.search(r' - (\w+)', info)
        category = category_match.group(1) if category_match else None
        
        size = re.search(r'\((\d+\.\d+)\s*"', info)
        size = size.group(1) if size else None
        
        space = re.search(r'(\d+)\s+(GB|TB)', info)
        space = space.group(1) + " " + space.group(2) if space else None
        
        color = re.split(r',\s*(?=\w)', info)[-1].rstrip(')')
        
        price = "".join([i.text for i in phone.find_elements(By.CSS_SELECTOR, 
                                                                 'div.price > span')])
        price = price.strip("-")
        
        condition = 'new'
        
        link_element = phone.find_element(By.CSS_SELECTOR, 'a.photo')
        phone_url = link_element.get_attribute('href')
        
        
        phone_features = {
            "page" : i,
            "brand": brand,
            "model": model,
            "category": category,
            "size": size,
            "space": space,
            "color": color,
            "price": price,
            "source": phone_url,
            "condition": condition,
            "date": pd.to_datetime(datetime.today().strftime('%Y-%m-%d'))
        }

        
        phone_list_page.append(phone_features)
    
    
    phone_list.extend(phone_list_page)    
    
    
    ### going inside each phone page
    for phone in phone_list_page:
    
        driver.get(phone['source'])
        
        ## catching broken pages
        try:
            body = driver.find_element(By.CSS_SELECTOR, "body > *")
        except NoSuchElementException:
            print(phone['source'])
            print("page broken")
            continue
        
        time.sleep(3)
        try:
            
            article_number = driver.find_element(By.CSS_SELECTOR, 
                                       'dl.group > dd > span[itemprop]').text

            rating = driver.find_element(By.CSS_SELECTOR, 
                                       '.bv_numReviews_text').text

            n_of_reviews = driver.find_element(By.CSS_SELECTOR, 
                                       'div[itemprop="ratingValue"]').text

            phone['article_number'] = article_number
            phone['rating'] = rating
            phone['n_of_reviews'] = n_of_reviews
        
        # catching phones without one of the info above
        except NoSuchElementException as e:
            print(f" {phone['source'] =}")
            print(e)
            
        
        #Print or use the extracted information

        
    if not smartphones:
        break
    
    time.sleep(3)
    
    i += 1
   

https://www.mediamarkt.ch/de/product/_samsung-galaxy-a54-5g-2173981.html
page broken
 phone['source'] ='https://www.mediamarkt.ch/de/product/_samsung-galaxy-a15-4g-2221377.html'
Message: Unable to locate element: div[itemprop="ratingValue"]; For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
RemoteError@chrome://remote/content/shared/RemoteError.sys.mjs:8:8
WebDriverError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:191:5
NoSuchElementError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:509:5
dom.find/</<@chrome://remote/content/shared/DOM.sys.mjs:136:16

 phone['source'] ='https://www.mediamarkt.ch/de/product/_samsung-galaxy-a15-4g-2221378.html'
Message: Unable to locate element: div[itemprop="ratingValue"]; For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace

In [43]:
driver.quit()

### list with all phones

In [14]:
phone_list

[{'page': 1,
  'brand': 'APPLE',
  'model': 'iPhone 13 ',
  'category': 'Smartphone',
  'size': '6.1',
  'space': '128 GB',
  'color': 'Midnight',
  'price': '536.',
  'source': 'https://www.mediamarkt.ch/de/product/_apple-iphone-13-2091423.html',
  'condition': 'new',
  'date': Timestamp('2024-03-22 00:00:00'),
  'article_number': '',
  'rating': '(77)',
  'n_of_reviews': '4.6'},
 {'page': 1,
  'brand': 'APPLE',
  'model': 'iPhone 15 Pro Max ',
  'category': 'Smartphone',
  'size': '6.7',
  'space': '256 GB',
  'color': 'Natural Titanium',
  'price': '1149.',
  'source': 'https://www.mediamarkt.ch/de/product/_apple-iphone-15-pro-max-2203429.html',
  'condition': 'new',
  'date': Timestamp('2024-03-22 00:00:00'),
  'article_number': '',
  'rating': '(30)',
  'n_of_reviews': '4.6'},
 {'page': 1,
  'brand': 'APPLE',
  'model': 'iPhone 15 ',
  'category': 'Smartphone',
  'size': '6.1',
  'space': '128 GB',
  'color': 'Black',
  'price': '719.',
  'source': 'https://www.mediamarkt.ch/de/pr

### transforming in a dataframe

In [16]:
df = pd.DataFrame(phone_list)
df

,page,brand,model,category,size,space,color,price,source,condition,date,article_number,rating,n_of_reviews
0,1,APPLE,iPhone 13,Smartphone,6.1,128 GB,Midnight,536.,https://www.mediamarkt.ch/de/product/_apple-ip...,new,2024-03-22,,(77),4.6
1,1,APPLE,iPhone 15 Pro Max,Smartphone,6.7,256 GB,Natural Titanium,1149.,https://www.mediamarkt.ch/de/product/_apple-ip...,new,2024-03-22,,(30),4.6
2,1,APPLE,iPhone 15,Smartphone,6.1,128 GB,Black,719.,https://www.mediamarkt.ch/de/product/_apple-ip...,new,2024-03-22,2203371,(23),4.8
3,1,APPLE,iPhone 15 Pro Max,Smartphone,6.7,256 GB,Black Titanium,1129.,https://www.mediamarkt.ch/de/product/_apple-ip...,new,2024-03-22,2203426,(11),4.9
4,1,APPLE,iPhone 15,Smartphone,6.1,256 GB,Black,819.,https://www.mediamarkt.ch/de/product/_apple-ip...,new,2024-03-22,2203376,(4),5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
360,16,APPLE,iPhone 13,Smartphone,6.1,128 GB,Pink,549.,https://www.mediamarkt.ch/de/product/_apple-ip...,new,2024-03-22,,(2),5.0
361,16,XIAOMI,13 Pro,Smartphone,6.73,256 GB,Ceramic White,1161.,https://www.mediamarkt.ch/de/product/_xiaomi-1...,new,2024-03-22,NaN,NaN,NaN
362,16,FAIRPHONE,4 5G,Smartphone,6.3,256 GB,Grau,556.,https://www.mediamarkt.ch/de/product/_fairphon...,new,2024-03-22,NaN,NaN,NaN
363,16,XIAOMI,13,Smartphone,6.36,256 GB,Schwarz,893.,https://www.mediamarkt.ch/de/product/_xiaomi-1...,new,2024-03-22,,(6),4.2


In [18]:
info_phone
# Extract article number
article_number_match = re.search(r'Artikelnummer:\n(\d+)', info_phone)
article_number = article_number_match.group(1) if article_number_match else None

# Extract rating
rating_match = re.search(r'(\d+\.\d+)\s+out\s+of\s+5\s+stars', info_phone)
rating = rating_match.group(1) if rating_match else None

# Extract number of reviews
reviews_match = re.search(r'Read\s+(\d+)\s+Reviews', info_phone)
reviews = reviews_match.group(1) if reviews_match else None

print(article_number)  # Output: 2203429
print(rating)          # Output: 4.6
print(reviews)         # Output: 30

2203429
4.6
30


In [216]:
import re

# Example text containing delivery information
delivery_info = 'Bestellbar, Auslieferung in 3-1 Werktagen'

# Define a regular expression pattern to match numbers or number ranges
pattern = r'\b(\d+)(?:-(\d+))?\b'

# Find all matches in the text
matches = re.findall(pattern, delivery_info)

# Initialize a variable to store the maximum number
max_number = 0

# Process each match
for match in matches:
    if len(match) == 1:  # Only one number found
        number = int(match[0])
        # Update max_number if the current number is greater
        max_number = max(max_number, number)
    else:  # Range of numbers found
        start = int(match[0])
        end = int(match[1])
        # Update max_number with the highest number from the range
        max_number = max(max_number, int(start), int(end))

print(max_number)  # Output the highest number


3


In [45]:
num_nan = df.isna().sum().sum()
num_nan

2139

In [192]:
nan_per_column = df.isna().sum()

# Alternatively, you can use isnull() instead of isna()
# nan_per_column = df.isnull().sum()

print("Number of NaN values per column:")
print(nan_per_column)

Number of NaN values per column:
deviceid            0
article_number    353
page                0
brand               0
model               0
category            0
condition           0
size               11
space               9
color               0
rating            353
n_of_reviews      353
delivery_time     353
price               0
source              0
date                0
dtype: int64


Problems i had: kicked using while   
problems i had: none values for space and it was because some phones are already 1 tb

In [66]:
unique_values_per_column = df.apply(lambda x: x.unique())

# Print the unique values for each column
print(unique_values_per_column)

deviceid          [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...
article_number                                                [nan]
page              [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...
brand             [APPLE, XIAOMI, SAMSUNG, MOTOROLA, NOTHING, FA...
model             [iPhone 15 Pro Max , iPhone 13 , iPhone 15 , i...
category                     [Smartphone, Klapphandy, Mobiltelefon]
condition                                                     [nan]
size              [6.7, 6.1, 6.74, 6.8, 6.4, 4.7, 6.5, 6.52, 6.6...
space             [256 GB, 128 GB, 512 GB, 32 GB, 64 GB, None, 8...
color             [Natural Titanium, Midnight, Black, Black Tita...
rating                                                        [nan]
n_of_reviews                                                  [nan]
delivery_time                                                 [nan]
price             [1149., 536., 719., 1129., 819., 619., 629., 1...
source            [https://www.mediamarkt.ch/de/

In [177]:
df['price'] = df['price'].astype(float)

In [179]:
df.sort_values(by=['price'], ascending=False).reset_index()

,index,deviceid,article_number,page,brand,model,category,condition,size,space,color,rating,n_of_reviews,delivery_time,price,source,date
0,257,258,NaN,11,SAMSUNG,Galaxy Z Fold4,Smartphone,NaN,7.6,256 GB,Phantom Black,NaN,NaN,NaN,1907.00,https://www.mediamarkt.ch/de/product/_samsung-...,2024-03-20
1,123,124,NaN,6,SAMSUNG,Galaxy Z Fold5,Smartphone,NaN,7.6,512 GB,Icy Blue,NaN,NaN,NaN,1778.00,https://www.mediamarkt.ch/de/product/_samsung-...,2024-03-20
2,261,262,NaN,11,SAMSUNG,Galaxy Z Fold5,Smartphone,NaN,7.6,256 GB,Phantom Black,NaN,NaN,NaN,1759.00,https://www.mediamarkt.ch/de/product/_samsung-...,2024-03-20
3,88,89,NaN,4,SAMSUNG,Galaxy Z Fold5,Smartphone,NaN,7.6,512 GB,Phantom Black,NaN,NaN,NaN,1710.00,https://www.mediamarkt.ch/de/product/_samsung-...,2024-03-20
4,263,264,NaN,11,SAMSUNG,Galaxy Z Fold5,Smartphone,NaN,7.6,256 GB,Cream,NaN,NaN,NaN,1693.00,https://www.mediamarkt.ch/de/product/_samsung-...,2024-03-20
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
348,44,45,NaN,2,XIAOMI,Redmi 9A,Smartphone,NaN,6.53,32 GB,Granite Grey,NaN,NaN,NaN,79.95,https://www.mediamarkt.ch/de/product/_xiaomi-r...,2024-03-20
349,27,28,NaN,2,XIAOMI,Redmi A2,Smartphone,NaN,6.52,32 GB,Schwarz,NaN,NaN,NaN,79.95,https://www.mediamarkt.ch/de/product/_xiaomi-r...,2024-03-20
350,188,189,NaN,8,DORO,5860,Mobiltelefon,NaN,None,None,DORO 5860 - Mobiltelefon (Schwarz/Weiss,NaN,NaN,NaN,79.95,https://www.mediamarkt.ch/de/product/_doro-586...,2024-03-20
351,258,259,NaN,11,WIKO,Y82,Smartphone,NaN,6.1,32 GB,Dark Blue,NaN,NaN,NaN,70.70,https://www.mediamarkt.ch/de/product/_wiko-y82...,2024-03-20


In [182]:
df.dtypes

deviceid                  object
article_number            object
page                      object
brand                     object
model                     object
category                  object
condition                 object
size                      object
space                     object
color                     object
rating                    object
n_of_reviews              object
delivery_time             object
price                    float64
source                    object
date              datetime64[ns]
dtype: object